In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [2]:
class Apriori:
    def __init__(self, min_supp, min_conf, products, transactions):
        self.min_supp = min_supp
        self.min_conf = min_conf
        self.products = products
        self.transactions = transactions
        self.trans_len = len(transactions)

    def occurence(self, p):
        occ = 0
        for t in self.transactions:
            occ += int(p in t)
        return occ/self.trans_len

    def supp(self, s):
        occ = 0
        for t in self.transactions:
            occ += int(s.issubset(t))
        return occ/self.trans_len
    
    def confidance(self, rule):
        return self.supp(rule[0] | rule[1])/self.supp(rule[0])
    
    def makeFrequentSets(self):
        F = list()
        F.append(frozenset({frozenset({p}) for p in self.products  if self.occurence(p) >= self.min_supp}))
        i = 1
#         print(F[0])
        while len(F[i-1]) > 0:
            C = { frozenset(L1 | L2) for L1 in F[i-1] for L2 in F[i-1] if len(L1 | L2) == i+1 and self.supp(L1 | L2) >= self.min_supp}
#             print(i,C)
            F.append(C)
            i += 1
        self.frequent_sets = list(itertools.chain.from_iterable(F))
        return self.frequent_sets
    
    def makeAssesRules(self):
        R = list()
        R.append({ (s,frozenset()) for s in self.frequent_sets})
        i = 1
        while len(R[i-1]) > 0:
            C = {(X[0] - {pre},X[1] | {pre}) for X in R[i-1] for pre in X[0] if self.confidance((X[0] - {pre},X[1] | {pre})) >= self.min_conf }
#             print(C)
            R.append(C)
            i += 1
        self.asses_rules = list(itertools.chain.from_iterable(R))
        return self.asses_rules
        
    def lift(self, rule):
        return self.confidance(rule)/self.supp(rule[1])
    
    def liftFilter(self, val):
        return [r for r in self.asses_rules if self.lift(r) >= val]
    
    def leverage(self, rule):
        return self.supp(rule[0] | rule[1]) - self.supp(rule[0]) * self.supp(rule[1])
    
    def leverageFilter(self, val):
        return [r for r in self.asses_rules if self.leverage(r) >= val]

In [ ]:
allTransactions = [set([int(id) for id in line.split()]) for line in open('/home/dominik/Dokumenty/Studia/Data-mining/Lista6-assiessing_rules/Lista06Dane3/')]

# data = pd.read_csv('/home/dominik/Dokumenty/Studia/Data-mining/Lista6-assiessing_rules/apriori/1000/1000-out1.csv', header=None)